In [1]:
import os
import sys
# 如果当前代码文件运行测试需要加入修改路径，避免出现后导包问题
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))

PYSPARK_PYTHON = "/miniconda2/envs/reco_sys/bin/python"
# 当存在多个版本时，不指定很可能会导致出错
os.environ["PYSPARK_PYTHON"] = PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"] = PYSPARK_PYTHON

from offline import SparkSessionBase
import pyhdfs
import time


class UpdateUserProfile(SparkSessionBase):
    """离线相关处理程序
    """
    SPARK_APP_NAME = "updateUser"
    ENABLE_HIVE_SUPPORT = True

    SPARK_EXECUTOR_MEMORY = "7g"

    def __init__(self):

        self.spark = self._create_spark_session()

uup = UpdateUserProfile()

In [3]:
# - 2、读取固定时间内的用户行为日志
uup.spark.sql("use profile")
user_action = uup.spark.sql("select actionTime, readTime, channelId, param.articleId, param.algorithmCombine, param.action, param.userId from user_action where dt>='2019-04-01'")
user_action.show()



+-------------------+--------+---------+--------------------+----------------+--------+-------------------+
|         actionTime|readTime|channelId|           articleId|algorithmCombine|  action|             userId|
+-------------------+--------+---------+--------------------+----------------+--------+-------------------+
|2019-04-09 07:13:25|        |        0|[15716, 19171, 13...|              C2|exposure|1103195673450250240|
|2019-04-09 07:13:49|        |       18|               19171|              C2|   click|1103195673450250240|
|2019-04-09 07:14:02|   12238|       18|               19171|              C2|    read|1103195673450250240|
|2019-04-09 07:14:04|        |       18|               13797|              C2|   click|1103195673450250240|
|2019-04-09 07:14:27|   21820|       18|               13797|              C2|    read|1103195673450250240|
|2019-04-09 07:13:25|        |        0|[15716, 19171, 13...|              C2|exposure|1103195673450250240|
|2019-04-09 07:13:49|       

In [5]:
# - 3、进行用户日志数据处理
def _compute(row):
    
    _list = []
    if row.action == 'exposure':
        for article_id in eval(row.articleId):
            # 用户ID跟文章ID拼接一个样本
            # ["user_id", "action_time","article_id", "channel_id", "shared", "clicked", "collected", "exposure", "read_time"]
            _list.append([row.userId, row.actionTime, article_id, row.channelId, False, False, False, True, row.readTime])
        return _list
    else:
        class Temp(object):
            shared = False
            clicked = False
            collected = False
            read_time = ""
        
        _tp = Temp()
        if row.action == 'click':
            _tp.clicked = True
        elif row.action == 'share':
            _tp.shared = True
        elif row.action == 'collect':
            _tp.collected = True
        elif row.action == 'read':
            _tp.clicked = True
        else:
            pass
        
        _list.append([row.userId, row.actionTime, int(row.articleId), row.channelId, _tp.shared, _tp.clicked, _tp.collected, True, row.readTime])
        
        return _list
        

_res = user_action.rdd.flatMap(_compute)

user_action_basic = _res.toDF(["user_id", "action_time","article_id", "channel_id", "shared", "clicked", "collected", "exposure", "read_time"])

In [6]:
user_action_basic.show()

+-------------------+-------------------+----------+----------+------+-------+---------+--------+---------+
|            user_id|        action_time|article_id|channel_id|shared|clicked|collected|exposure|read_time|
+-------------------+-------------------+----------+----------+------+-------+---------+--------+---------+
|1103195673450250240|2019-04-09 07:13:25|     15716|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04-09 07:13:25|     19171|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04-09 07:13:25|     13797|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04-09 07:13:25|     17511|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04-09 07:13:25|     18795|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04-09 07:13:25|     18156|         0| false|  false|    false|    true|         |
|1103195673450250240|2019-04

In [8]:
# - 4、存储到user_article_basic表中
old = uup.spark.sql("select * from user_article_basic")

In [9]:
new = old.unionAll(user_action_basic)

In [ ]:
# new.registerTempTable('temptable')

# uup.spark.sql("insert overwrite table user_article_basic select user_id, max(action_time) as action_time, "
#         "article_id, max(channel_id) as channel_id, max(shared) as shared, max(clicked) as clicked, "
#         "max(collected) as collected, max(exposure) as exposure, max(read_time) as read_time from temptable "
#         "group by user_id, article_id")

In [10]:
# 用户画像的关键词获取以及权重计算
# 1、读取user_article_basic表，合并行为表与文章画像中的主题词
uup.spark.sql("use profile")
user_basic = uup.spark.sql("select * from user_article_basic").drop('channel_id')


In [11]:
user_basic.show()

+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+
|            user_id|        action_time|         article_id|shared|clicked|collected|exposure|read_time|
+-------------------+-------------------+-------------------+------+-------+---------+--------+---------+
|1105045287866466304|2019-03-11 18:13:45|              14225| false|  false|    false|    true|         |
|1106476833370537984|2019-03-15 16:46:50|              14208| false|  false|    false|    true|         |
|1109980466942836736|2019-03-25 08:50:36|              19233| false|  false|    false|    true|         |
|1109980466942836736|2019-03-25 16:40:37|              44737| false|  false|    false|    true|         |
|1109993249109442560|2019-03-25 09:39:48|              17283| false|  false|    false|    true|         |
|1111189494544990208|2019-03-28 17:02:35|              19322| false|  false|    false|    true|         |
|1111524501104885760|2019-03-29 15:04:27|     

In [12]:
#读取文章画像
uup.spark.sql("use article")
article_topic = uup.spark.sql("select  article_id, channel_id, topics from article_profile")

In [13]:
article_topic.show()

+----------+----------+--------------------+
|article_id|channel_id|              topics|
+----------+----------+--------------------+
|        26|        17|[Electron, 全自动, 产...|
|        29|        17|[WebAssembly, 影音,...|
|       474|        17|[textAlign, borde...|
|       964|        11|[protocol, RMI, d...|
|      1677|        17|[spritesmith, ico...|
|      1697|         6|[nav, 样式, width, ...|
|      1806|        17|[声明, word, 容器, Ex...|
|      1950|        17|[app, scss, koala...|
|      2040|        17|[宽度, 媒体, width, r...|
|      2214|        11|[Cyber, 语言, 黑客, 知...|
|      2250|         6|[宽度, cal, 阶梯, 页面,...|
|      2453|        13|[__, CNN, logisti...|
|      2509|        13|[池化, CNN, 卷积神经网络,...|
|      2529|        17|[标题栏, 定义, 嵌套, hea...|
|      2927|         6|[季风, 圆角, bezier, ...|
|      3091|         6|[Chrome, react, 工...|
|      3506|        17|[cond, AJAX, 实心, ...|
|      3764|        15|[__, 语言, 原型链, obj...|
|      4590|        19|[println, Class, ...|
|      482

In [14]:
user_topic = user_basic.join(article_topic, on=['article_id'], how='left')

In [15]:
user_topic.show()

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|              topics|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------------------+
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|     13401|1114864237131333632|2019-04-09 16:39:51| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|     13401|1106396183141548032|2019-03-28 10:58:20| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|     13401|1109994594201763840|2019-03-26 15:03:58| false|  false|    false|    true|         |        18|[补码, 字符串, 李白, typ...|
|     14805|1105045287866466304|2019-03-11 18:15:48| false|  false|    false|    true|         | 

In [16]:
import pyspark.sql.functions as F

user_topic = user_topic.withColumn('topic', F.explode('topics')).drop('topics')


In [17]:
user_topic.show()

+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|article_id|            user_id|        action_time|shared|clicked|collected|exposure|read_time|channel_id|   topic|
+----------+-------------------+-------------------+------+-------+---------+--------+---------+----------+--------+
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      补码|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|     字符串|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      李白|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|    type|
|     13401|                 10|2019-03-06 10:06:12| false|  false|    false|    true|         |        18|      元素|
|     13401|                 10|2019-03-06 10:06:12| false|  fal

In [21]:
def compute_user_label_weights(partitions):
    """# 计算用户关键词权重
    """
    weightsOfaction = {
        "read_min": 1,
        "read_middle": 2,
        "collect": 2,
        "share": 3,
        "click": 5
    }
    
    # 导入包
    from datetime import datetime
    import numpy as np
    
    
    # 循环每个用户对应每个关键词处理
    for row in partitions:
        
        # 计算时间系数
        t = datetime.now() - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
        alpha = 1 / (np.log(t.days + 1) + 1)
        
        # 判断一下这个关键词对应的操作文章时间大小的权重处理
        if row.read_time  == '':
            read_t = 0
        else:
            read_t = int(row.read_time)
        
        # 阅读时间的行为分数计算出来
        read_score = weightsOfaction['read_middle'] if read_t > 1000 else weightsOfaction['read_min']
        
        # 计算row.topic的权重
        weights = alpha * (row.shared * weightsOfaction['share'] + row.clicked * weightsOfaction['click'] +
                          row.collected * weightsOfaction['collect'] + read_score)
        
        # user_profilehbase表
        #        with pool.connection() as conn:
#            table = conn.table('user_profile')
#            table.put('user:{}'.format(row.user_id).encode(),
#                      {'partial:{}:{}'.format(row.channel_id, row.topic).encode(): json.dumps(
#                          weigths).encode()})
#            conn.close()
        


user_topic.foreachPartition(compute_user_label_weights)